In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
df=spark.read.option("multiline","true").json('/content/Py-SparkcodetoFlattenJSON.json')
df.show()

+--------------------+---------+
|               feeds|totalFeed|
+--------------------+---------+
|[[0, 0, 2020-01-0...|      125|
+--------------------+---------+



In [4]:
def all_columns_apart_from_the_particular_column(df,name_string):
  ms=df.schema.fields;
  ag=[];
  for i in ms:
    if(i.name!=name_string):
      ag.append(i.name);
  return ag;

In [5]:
def expand_structure(df,dfstruct,al):
  ag=[];
  for i in dfstruct:
    ag.append(al+"."+i.name);
  return ag;

In [6]:
def flat_df(df):
  all_raw_fields=df.schema.fields;
  for i in all_raw_fields:
    if (str(i.dataType)[0:9])=="ArrayType":
      column_list=all_columns_apart_from_the_particular_column(df,i.name);
      explosion="explode("+i.name+")" "as "+"Exploded_"+i.name;
      column_list.append(explosion);
      df=df.selectExpr(column_list);
    elif (str(i.dataType)[0:10])=="StructType":
      column_list=all_columns_apart_from_the_particular_column(df,i.name);
      dfstruct=df.schema[i.name].dataType.fields;
      all_fields_within_structure=expand_structure(df,dfstruct,i.name);
      proper_name = [ x+" as "+ x.replace('.', "_") for x in all_fields_within_structure ];
      column_list.extend(proper_name)
      df=df.selectExpr(column_list);
  return df;

In [7]:
df1=flat_df(df);
df2=flat_df(df1);
df3=flat_df(df2);
df4=flat_df(df3);

In [8]:
df4.show()

+---------+-------------------+---------------------------+------------------------+--------------------------+---------------------------+-----------------+-------------------------+---------------------+------------------------+------------------+------------------+-----------------------+------------------------+------------------+-------------------+-----------------------------+--------------------+---------------------+-----------------------+-----------------------------------+--------------------------------+-------------------------------------+-------------------------------------------+----------------------------------------------+-------------------------------------+--------------------------------------------+--------------------------------------------+---------------------------------------+----------------------------------------+--------------------------------------+
|totalFeed|Exploded_feeds_code|Exploded_feeds_commentCount|Exploded_feeds_createdAt|Exploded_feeds_d

# **Main Code to Execute**

In [22]:
def encapsulation(df):
  ms=str(df.schema);
  if (("ArrayType" in ms)):
    processing=flat_df(df);
    processing.show();
    return encapsulation(processing);
  elif (ms.count("StructType")>1):
    processing=flat_df(df);
    processing.show()
    return encapsulation(processing);
  else:
    return (df);

In [23]:
ag=encapsulation(df)

+---------+--------------------+
|totalFeed|      Exploded_feeds|
+---------+--------------------+
|      125|[0, 0, 2020-01-02...|
|      125|[0, 2, 2020-01-02...|
|      125|[0, 0, 2020-01-02...|
|      125|[0, 7, 2020-01-02...|
|      125|[0, 0, 2020-01-02...|
|      125|[0, 0, 2020-01-02...|
|      125|[0, 2, 2020-01-02...|
|      125|[0, 1, 2020-01-02...|
|      125|[0, 0, 2020-01-02...|
|      125|[0, 2, 2020-01-01...|
|      125|[0, 1, 2020-01-01...|
|      125|[0, 1, 2020-01-01...|
|      125|[0, 1, 2020-01-01...|
|      125|[0, 2, 2020-01-01...|
|      125|[0, 0, 2020-01-01...|
|      125|[0, 0, 2020-01-01...|
|      125|[0, 0, 2020-01-01...|
|      125|[0, 0, 2020-01-01...|
|      125|[0, 0, 2020-01-01...|
|      125|[0, 0, 2020-01-01...|
+---------+--------------------+
only showing top 20 rows

+---------+-------------------+---------------------------+------------------------+--------------------------+---------------------------+-----------------+-------------------------

In [24]:
ag.show()

+---------+-------------------+---------------------------+------------------------+--------------------------+---------------------------+-----------------+-------------------------+---------------------+------------------------+------------------+------------------+-----------------------+------------------------+------------------+-------------------+-----------------------------+--------------------+---------------------+-----------------------+-----------------------------------+--------------------------------+-------------------------------------+-------------------------------------------+----------------------------------------------+-------------------------------------+--------------------------------------------+--------------------------------------------+---------------------------------------+----------------------------------------+--------------------------------------+
|totalFeed|Exploded_feeds_code|Exploded_feeds_commentCount|Exploded_feeds_createdAt|Exploded_feeds_d

In [26]:
ag.count(),len(ag.columns)

(296, 31)